 # EDA #

Goal: explore and examine **`sp500_daily_close.parquet`**

Observations:
1. There are no NaN values in the columns.
2. The columns have efficient datatypes Date: datetime64[ns], Close: float32, Symbol: category.
3. The value ranges are:
    * Date: 1962-01-02 00:00:00 → 2025-07-31 00:00:00
    * Close: 0.00 → 10969.55
    * Tickers: 500 (3 missing)
        * The last three tickers ZBRA, ZBH, and ZTS have not been downloaded because we have exceeded the daily hits limit
        * I will need to download the last three tickers tomorrow and merge the parquet datasets
4. The parquet df has no duplicates.
5. The Stooq and Yahoo finance price histories approximately coincide. The examined prices are different in cents.
6. The Stooq Close prices are highly probably adjusted for dividends, stock splits, and spin-offs.
    * The assumed adjusted yfinance prices seem to coincide with Stooq prices and show no jumps in either source around known stock splits. For example, we found adjusted Stooq and yfinance priced for APPL split 4:1 on Aug 31, 2020.
7. All close prices are positive.
8. The time series seems continuous. For example, AAPL is missing 366 trading days over decades which is normal because of US-specific holidays, halted trading, Stooq missing data, etc.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
# Load Parquet file
df = pd.read_parquet("../data/raw/sp500_daily_close.parquet")


In [8]:
# 1. Columns
print("Columns:")
print(df.columns.tolist())

Columns:
['Date', 'Close', 'Symbol']


In [9]:
# 2. NaN values
print("\nMissing values per column:")
print(df.isna().sum())


Missing values per column:
Date      0
Close     0
Symbol    0
dtype: int64


In [10]:
# 3. Data types
print("\nData types:")
print(df.dtypes)


Data types:
Date      datetime64[ns]
Close            float32
Symbol          category
dtype: object


In [11]:
# 4. Value Ranges
print("\nDate Range:")
print(f"{df['Date'].min()} → {df['Date'].max()}")

print("\nClose Price Range:")
print(f"{df['Close'].min():.2f} → {df['Close'].max():.2f}")

print("\nUnique Tickers:", df['Symbol'].nunique())


Date Range:
1962-01-02 00:00:00 → 2025-08-01 00:00:00

Close Price Range:
0.00 → 10969.55

Unique Tickers: 503


In [12]:
# ----------------------------------------------------------------------
# CONFIGURABLE PARAMETERS
# ----------------------------------------------------------------------
WIKI_SP500_URL      = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
STOOQ_URL_TEMPLATE  = "https://stooq.com/q/d/l/?s={symbol}.us&i=d"
OUT_PATH            = "data/raw/sp500_daily_close500.parquet"
REQUEST_TIMEOUT     = 20        # seconds per HTTP request
SLEEP_BETWEEN_CALLS = 0.3       # polite delay between calls
# ----------------------------------------------------------------------


def get_sp500_tickers() -> list[str]:
    """Return a list of S&P 500 tickers (upper-case, '.'→'-')."""
    tables = pd.read_html(WIKI_SP500_URL)
    return (
        tables[0]["Symbol"]
        .str.replace(".", "-", regex=False)
        .str.upper()
        .tolist()
    )

tickers = get_sp500_tickers()
dataset_tickers = df['Symbol'].unique()

missing = list(set(tickers) - set(dataset_tickers))

print("\nThe dataset misses the following tickers:", missing)


The dataset misses the following tickers: []


In [22]:
import requests, io

def fetch_stooq_csv(symbol: str) -> pd.DataFrame | None:
    url = STOOQ_URL_TEMPLATE.format(symbol=symbol.lower())
    print("URL:", url)
    try:
        r = requests.get(url, timeout=REQUEST_TIMEOUT)
        print("Status code:", r.status_code)
        print("Response (first 300 chars):", r.text[:300])

        if r.status_code != 200 or "Exceeded the daily hits limit" in r.text:
            return None

        df = pd.read_csv(io.StringIO(r.text), usecols=["Date", "Close"])
        df["Symbol"] = symbol
        return df
    except Exception as e:
        print("Error:", e)
        return None

print(fetch_stooq_csv("A"))

URL: https://stooq.com/q/d/l/?s=a.us&i=d
Status code: 200
Response (first 300 chars): Exceeded the daily hits limit
None


In [13]:
# Total duplicate rows (all columns match)
dup_rows = df.duplicated().sum()
print(f"Total duplicate rows: {dup_rows}")

Total duplicate rows: 0


In [14]:
# Check some well-known tickers (e.g., AAPL, TSLA)
sample_tickers = ['AAPL', 'TSLA', 'AMZN', 'GOOGL']

for ticker in sample_tickers:
    sub = df[df['Symbol'] == ticker].sort_values('Date')
    sub['pct_change'] = sub['Close'].pct_change()

    large_moves = sub[sub['pct_change'].abs() > 0.3]  # >30% daily change
    if not large_moves.empty:
        print(f"\n⚠️ Possible unadjusted data for {ticker} (split/dividend events?):")
        print(large_moves[['Date', 'Close', 'pct_change']])
    else:
        print(f"{ticker}: no large jumps detected.")


⚠️ Possible unadjusted data for AAPL (split/dividend events?):
            Date     Close  pct_change
9724  1997-08-06  0.198047    0.332528
10518 2000-09-29  0.387655   -0.518474

⚠️ Possible unadjusted data for TSLA (split/dividend events?):
              Date    Close  pct_change
3115406 2010-06-29  1.59267    0.405301

⚠️ Possible unadjusted data for AMZN (split/dividend events?):
             Date   Close  pct_change
229569 2001-04-09  0.5590    0.335723
229718 2001-11-14  0.4745    0.301783
229725 2001-11-26  0.6105    0.344714
GOOGL: no large jumps detected.


In [26]:
df_appl = df[df['Symbol'] == 'AAPL'].set_index('Date').sort_index()
print(df_appl.loc['2020-08-20':'2020-09-02'][['Close']])

                 Close
Date                  
2020-08-20  115.332001
2020-08-21  121.264999
2020-08-24  122.724998
2020-08-25  121.709000
2020-08-26  123.374001
2020-08-27  121.886002
2020-08-28  121.698997
2020-08-31  125.829002
2020-09-01  130.834000
2020-09-02  128.115005


In [28]:
import yfinance as yf

yf_data = yf.download("AAPL", start="2020-08-20", end="2020-09-02")
print(yf_data[["Close"]])

C:\Users\strakad\AppData\Local\Temp\ipykernel_31448\4101677522.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf_data = yf.download("AAPL", start="2020-08-20", end="2020-09-02")
[*********************100%***********************]  1 of 1 completed

Price            Close
Ticker            AAPL
Date                  
2020-08-20  115.178764
2020-08-21  121.114212
2020-08-24  122.562767
2020-08-25  121.557312
2020-08-26  123.210358
2020-08-27  121.737457
2020-08-28  121.540260
2020-08-31  125.661964
2020-09-01  130.667374


In [15]:
bad_prices = df[df['Close'] <= 0]
print(f"Bad price entries: {len(bad_prices)}")

Bad price entries: 0


In [38]:
# Example: check how many trading days AAPL is missing
all_dates = pd.Series(pd.date_range(df[df["Symbol"] == "AAPL"].Date.min(), df[df["Symbol"] == "AAPL"].Date.max(), freq='B'))  # business days
aapl_dates = df[df['Symbol'] == 'AAPL']['Date']

missing = all_dates[~all_dates.isin(aapl_dates)]
print(f"AAPL missing {len(missing)} trading days")

AAPL missing 366 trading days


In [16]:
all_dates = pd.Series(pd.date_range(df[df["Symbol"] == "AAPL"].Date.min(), df[df["Symbol"] == "AAPL"].Date.max(), freq='B'))
print(f"Total trading days in date range: {len(all_dates)}")

Total trading days in date range: 10670


In [34]:
aapl = df[df['Symbol'] == 'AAPL']
print(f"AAPL data points: {aapl.shape[0]}")


AAPL data points: 10304


In [35]:
print(aapl['Date'].min(), "→", aapl['Date'].max())

1984-09-07 00:00:00 → 2025-07-31 00:00:00


In [3]:
import pyarrow.parquet as pq
import pyarrow as pa

# Read the Parquet files into tables
table1 = pq.read_table("data/raw/sp500_daily_close500.parquet")
table2 = pq.read_table("data/raw/sp500_daily_close3.parquet")

# Cast Symbol column in table2 to match table1 (int16 dictionary)
symbol_type = table1.schema.field("Symbol").type
table2 = table2.cast(table1.schema)

# Concatenate tables
combined_table = pa.concat_tables([table1, table2])

# Write the result
pq.write_table(combined_table, "data/raw/sp500_daily_close.parquet")
